In [1]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt,butter, lfilter
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split,ShuffleSplit
from sklearn.preprocessing import label_binarize,scale
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize,StandardScaler

In [2]:
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()

from eegPinelineDesign import CenterAtPeakOfWindow,Threshold_test,spindle_overlapping_test,used_windows_check,cut_segments


In [3]:
import pickle
with open('single subject.p','rb') as handle:
    result = pickle.load(handle)

In [4]:
result.keys()

dict_keys(['nonspindle time', 'non spindle', 'spindle time', 'spindle'])

In [5]:
result['spindle'].keys()

dict_keys(['suj10_d1final'])

In [6]:
fileName=list(result['spindle'].keys())[0]
channelList=['F3','F4','C3','C4','O1','O2']
X = [];Y=[]
for names in channelList:
    for item in result['spindle'][fileName][names]:
        if item.shape[1] == 3000:

            X.append(item[0,:])
            Y.append(1)
    for item in result['non spindle'][fileName][names]:
        if item.shape[1] == 3000:
            X.append(item[0,:])
            Y.append(0)

X=np.array(X);Y=np.array(Y)

In [7]:
X.shape,Y.shape

((1451, 3000), (1451,))

In [8]:
idx=np.arange(X.shape[0])
GG=np.random.choice(tuple(idx),len(idx),replace=False)
def shuffle(x):
    return sorted(x, key=lambda k: random.random())
GG = shuffle(shuffle(shuffle(shuffle(GG))))
XX=[];YY=[]
for idxx in GG:
    XX.append(X[idxx])
    YY.append(Y[idxx])

from sklearn.preprocessing import normalize
normal_X = normalize(XX)
X_train, X_test, y_train, y_test = train_test_split(normal_X, YY, test_size=0.20)

#print(y_train)

clf =LogisticRegression(penalty='l2',C=.1,tol=10e-9,fit_intercept=True,solver='liblinear',
                                             max_iter=10e7,multi_class='ovr',n_jobs=-1)



clf.fit(X_train,y_train)
print('logistic,1 fold, 80/20 train/test slipt')
print(classification_report(clf.predict(X_test),y_test))
print('within training data set',clf.score(X_train,y_train))

logistic,1 fold, 80/20 train/test slipt
             precision    recall  f1-score   support

          0       0.91      0.94      0.93       148
          1       0.94      0.91      0.92       143

avg / total       0.92      0.92      0.92       291

within training data set 0.927586206897


In [15]:
num=2
file_to_read,fileName=eegPinelineDesign.pick_sample_file(EDFfiles,n=num)
channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
raw_filter = eegPinelineDesign.load_data(file_to_read,channelList,11, 16)#spindle pass
raw_alpha=eegPinelineDesign.load_data(file_to_read,channelList,8, 12)#alpha pass
raw_spindle=eegPinelineDesign.load_data(file_to_read,channelList,11, 16)#spindle pass
raw_muscle=eegPinelineDesign.load_data(file_to_read,channelList,30, 40)#
channelList = ['F3','F4','C3','C4','O1','O2']
raw_filter.pick_channels(channelList)

Extracting edf Parameters from suj10_d1final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3601999  =      0.000 ...  3601.999 secs...
[done]
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 2 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    2, 7, 4, 4, 2
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 3 ICA components
Inverse transforming to PCA space
Reconstructing sensor space signals from 8 PCA components
Band-pass filtering from 11 - 16 Hz
Extracting edf Parameters from suj10_d1final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3601999  =      0.000 ...  3601.999 secs...
[done]
Ready.
Fitting ICA to data using 8 

<RawEDF  |  n_channels x n_times : 6 x 3602000>

In [16]:
time_label={};resolution = 0.02
for names in channelList:
    time_label[names]=[]

In [ ]:
centerPoint = 0+5;cnt = 0
while raw_filter.last_samp/1000 - centerPoint > 1.5:
    if (cnt % 500) == 0:
        print(centerPoint)
    for ii,names in enumerate(channelList):
    
        if Threshold_test(centerPoint,raw_alpha,raw_spindle,raw_muscle,ii):
            tempSegment,timeSpan=cut_segments(raw_filter,centerPoint,ii,windowsize = 1.5)
            predictedLabel = clf.predict(tempSegment[0,:3000])
            time_label[names].append([centerPoint,predictedLabel])
        else:
            time_label[names].append([centerPoint,0])

    centerPoint += resolution;cnt += 1
print('done')

5
14.999999999999787
24.999999999999574
34.99999999999989
45.00000000000146
55.00000000000302
65.00000000000422
75.00000000000223
85.00000000000024
94.99999999999825
104.99999999999626
114.99999999999427
124.99999999999228
134.99999999999525
145.00000000000037
155.00000000000549
165.0000000000106
175.00000000001572
185.00000000002083
195.00000000002595
205.00000000003106
215.00000000003618
225.0000000000413
235.0000000000464
245.00000000005153
255.00000000005664
265.00000000004894
275.00000000003985
285.00000000003075
295.00000000002166
305.00000000001256
315.00000000000347
324.9999999999944
334.9999999999853
344.9999999999762
354.9999999999671
364.999999999958
374.9999999999489
384.9999999999398
394.9999999999307
404.9999999999216
414.9999999999125
424.9999999999034
434.9999999998943
444.99999999988523
454.99999999987614
464.99999999986704
474.99999999985795
484.99999999984885
494.99999999983976